# The Iterable Operation Extensions

<--- Back

## Table of Contents for this notebook

- [The Iterable Operation Extensions](#The-Iterable-Operation-Extensions)
- [Table of Contents for this notebook](#Table-of-Contents-for-this-notebook)
- [Overview](#Overview)
- [IterEnumerateInstances](#IterEnumerateInstances)
- [IterEnumerateInstancePaths](#IterEnumerateInstancePaths)
- [IterAssociatorInstances](#IterAssociatorInstances)
- [IterAssociatorInstancePaths](#IterAssociatorInstancePaths)
- [IterReferenceInstances](#IterReferenceInstances)
- [IterReferenceInstancePaths](#IterReferenceInstancePaths)
- [IterQueryInstances](#IterQueryInstances)
- [Using list comprehensions with iter operations](#Using-list-comprehensions-with-iter-operations)
- [Using iter operations and forcing the use of traditional operations](#Using-iter-operations-and-forcing-the-use-of-traditional-operations)
- [Using iter operations and forcing the use of pull operations](#Using-iter-operations-and-forcing-the-use-of-pull-operations)


The iterable operation extensions (short: *iter operations*) are a set of methods added to [`pywbem.WBEMConnection`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection) class in pywbem version 0.10.0 to simplify the use of the pull vs. traditional operations (the original enumeration operations such as EnumerateInstances).

They simplify executing instance enumerations (EnumerateInstances, Associators, and References) by merging the traditional operations and pull operations so the client developer only has to code the iter operations and the Pywbem environment uses the Pull or traditional operations based on the server characteristics.

The purpose and function and usage of these WBEMConnection methods is defined in the [`Concepts section`] (https://pywbem.readthedocs.io/en/latest/concepts.html#iter-operations) of the online pywbem documentation.

## IterEnumerateInstances

The [`IterEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterEnumerateInstances) method requests the instances of a class from the server. It uses either the [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstances) if the server supports it or otherwise [`EnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstances).

The operation returns an iterator for instances, that is being processed in a for-loop.

In [ ]:
import pywbem

# Global variables used by all examples:
server = 'http://localhost'
username = 'user'
password = 'password'
namespace = 'root/cimv2'

classname = 'CIM_ComputerSystem'
max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    inst_iterator = conn.IterEnumerateInstances(classname,
                                                MaxObjectCount=max_obj_cnt)
    for inst in inst_iterator:
        print('path=%s' % inst.path)
        print(inst.tomof())
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

In this example (and the examples below), the connection has a default namespace set. This allows us to omit the namespace from the subsequent [`IterEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterEnumerateInstances) method.

This example and the following examples also shows exception handling with pywbem: Pywbem wraps any exceptions that are considered runtime errors, and raises them as subclasses of [`pywbem.Error`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.Error). Any other exceptions are considered programming errors. Therefore, the code above only needs to catch [`pywbem.Error`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.Error).

Note that the creation of the [`pywbem.WBEMConnection`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection) object in the code above does not need to be protected by exception handling; its initialization code does not raise any pywbem runtime errors.

## IterEnumerateInstancePaths

The [`IterEnumerateInstancePaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterEnumerateInstancePaths) method requests the instance paths of the instances of a class from the server. It uses either the [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstancePaths) if the server supports it or otherwise [`EnumerateInstanceNames()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstanceNames).

The operation returns an iterator for instance paths, that is being processed in a for-loop:

In [ ]:
# Global variables from first example are used

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    path_iterator = conn.IterEnumerateInstancePaths(classname,
                                                    MaxObjectCount=max_obj_cnt)
    for path in path_iterator:
        print('path=%s' % path)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

## IterAssociatorInstances

The [`IterAssociatorInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterAssociatorInstances) method requests the instances associated with a source instance from the server. It uses either the [`OpenAssociatorInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenAssociatorInstances) if the server supports it or otherwise [`Associators()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.Associators).

The operation returns an iterator for instances, that is being processed in a for-loop.

The following code executes [`EnumerateInstanceNames()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstanceNames) to enumerate instances of a class and selects the first instance to act as a source instance for the association operation:

In [ ]:
# Global variables from first example are used

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    # Find an instance path for the source end of an association.
    source_paths = conn.EnumerateInstanceNames(classname)
    if source_paths:
        inst_iterator = conn.IterAssociatorInstances(source_paths[0],
                                                     MaxObjectCount=max_obj_cnt)
        for inst in inst_iterator:
            print('path=%s' % inst.path)
            print(inst.tomof())
    else:
        print('%s class has no instances and therefore no associations', classname)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

## IterAssociatorInstancePaths

The [`IterAssociatorInstancePaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterAssociatorInstancePaths) method requests the instance paths of the instances associated with a source instance from the server. It uses either the [`OpenAssociatorInstancePaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenAssociatorInstancePaths) if the server supports it or otherwise [`AssociatorNames()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.AssociatorNames).

The operation returns an iterator for instance paths, that is being processed in a for-loop.

The following code executes [`EnumerateInstanceNames()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstanceNames) to enumerate instances of a class and selects the first instance to act as a source instance for the association operation:

In [ ]:
# Global variables from first example are used

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    # Find an instance path for the source end of an association.
    source_paths = conn.EnumerateInstanceNames(classname)
    if source_paths:
        path_iterator = conn.IterAssociatorInstancePaths(source_paths[0],
                                                         MaxObjectCount=max_obj_cnt)
        for path in path_iterator:
            print('path=%s' % path)
    else:
        print('%s class has no instances and therefore no instance associations', classname)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

## IterReferenceInstances

The [`IterReferenceInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterReferenceInstances) method requests the instances referencing a source instance from the server. It uses either the [`OpenReferenceInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenReferenceInstances) if the server supports it or otherwise [`References()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.References).

The operation returns an iterator for instances, that is being processed in a for-loop.

The following code executes [`EnumerateInstanceNames()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstanceNames) to enumerate instances of a class and selects the first instance to act as a source instance for the reference operation:

In [ ]:
# Global variables from first example are used

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    # Find an instance path for the source end of an association.
    source_paths = conn.EnumerateInstanceNames(classname)
    if source_paths:
        inst_iterator = conn.IterAssociatorInstances(source_paths[0],
                                                     MaxObjectCount=max_obj_cnt)
        for instance in inst_iterator:
            print('path=%s' % inst.path)
            print(inst.tomof())
    else:
        print('%s class has no instances and therefore no instance associations', classname)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

## IterReferenceInstancePaths

The [`IterReferenceInstancePaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterReferenceInstancePaths) method requests the instance paths of the instances referencing a source instance from the server. It uses either the [`OpenReferenceInstancePaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenReferenceInstances) if the server supports it or otherwise [`ReferenceNames()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.ReferenceNames).

The operation returns an iterator for instance paths, that is being processed in a for-loop.

The following code executes [`EnumerateInstanceNames()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstanceNames) to enumerate instances of a class and selects the first instance to act as a source instance for the reference operation:

In [ ]:
# Global variables from first example are used

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    # Find an instance path for the source end of an association.
    source_paths = conn.EnumerateInstanceNames(classname)
    if source_paths:
        path_iterator = conn.IterReferenceInstancePaths(source_paths[0],
                                                        MaxObjectCount=max_obj_cnt)
        for path in path_iterator:
            print('path=%s' % path)
    else:
        print('%s class has no instances and therefore no instance associations', classname)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

## IterQueryInstances

The [`IterQueryInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterQueryInstances) method requests the instances returned by a query from the server. It uses either the [`OpenQueryInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenQueryInstances) if the server supports it or otherwise [`ExecQuery()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.ExecQuery).

The operation returns an iterator for instances, that is being processed in a for-loop:

In [ ]:
# Global variables from first example are used

query_language = "DMTF:CQL"
query = 'Select * from Pywbem_Person'
max_obj_cnt = 10

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    inst_iterator = conn.IterQueryInstances(query_language, query,
                                            MaxObjectCount=max_obj_cnt)
    for inst in inst_iterator:
        print(inst.tomof())
    else:
        print('query has no result')
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

## Using list comprehensions with iter operations

As an alternative to the for-loop processing of the iterator returned by the `Iter...()` method, a list comprehension can be use to perform the entire processing sequence in a single statement:

In [ ]:
# Global variables from first example are used

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    paths = [path for path in conn.IterEnumerateInstancePaths(classname,
                                                              MaxObjectCount=max_obj_cnt)]
    print(*paths, sep='\n')
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

## Using iter operations and forcing the use of traditional operations

The following example forces the use of the traditional operations by setting the `use_pull_operations` flag on the connection to `False`:

In [ ]:
# Global variables from first example are used

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True,
                             use_pull_operations=False)

try:
    inst_iterator = conn.IterEnumerateInstances(classname,
                                                MaxObjectCount=max_obj_cnt)
    for inst in inst_iterator:
        print('path=%s' % inst.path)
        print(inst.tomof())
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

## Using iter operations and forcing the use of pull operations

The following example forces the use of the pull operations by setting the `use_pull_operations` flag on the connection to `True`. If the server does not support pull operations an exception "CIM_ERR_NOT_SUPPORTED" will be returned.

In [ ]:
# Global variables from first example are used

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True,
                             use_pull_operations=True)

try:
    inst_iterator = conn.IterEnumerateInstances(classname,
                                                MaxObjectCount=max_obj_cnt)
    for inst in inst_iterator:
        print('path=%s' % inst.path)
        print(inst.tomof())
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)